In [1]:
import torch
from torch import optim
from torchsummary import summary
from torch.utils.data import random_split, DataLoader
from torchvision.datasets import CIFAR100
from torchvision.transforms import v2
from pathlib import Path
import wandb
import sys
import os

sys.path.append("../src")

from trainer import Trainer
from module import ResNetModule
from utils import model_size, load_from_checkpoint
from callbacks import OverfitCallback, EarlyStoppingCallback
from logger import WandbLogger
from dataset import MapDataset
from models import ResNet18
import settings as s

In [2]:
data_path = Path("../data")
logs_path = Path("../logs")
logs_path.mkdir(exist_ok=True)

In [3]:
logger = WandbLogger(
    project_name=s.project_name,
    config={
        "model": s.model,
        "dataset": s.dataset,
        "max_epochs": s.max_epochs,
        "optimizer": s.optimizer,
        "lr_scheduler": s.lr_scheduler,
        "test_run": s.test_run
    },
    logs_path=logs_path,
    offline=s.wandb_offline
)

In [4]:
# cpu_count = os.cpu_count()
cpu_count = 7

dataset = CIFAR100(data_path, train=True, download=True)

train_dataset, val_dataset = random_split(
    dataset, [s.dataset["train_split"], s.dataset["val_split"]]
)

normalize_transforms = v2.Compose([
    # Normalize
    v2.ToImage(),
    v2.ToDtype(torch.float32, scale=True),
    v2.Normalize(mean=(0.4914, 0.4822, 0.4465), std=(0.2023, 0.1994, 0.2010))
])

train_transforms = v2.Compose([
    v2.RandomCrop(size=(32, 32), padding=4, padding_mode='reflect'),
    v2.RandomHorizontalFlip(),
    v2.RandomVerticalFlip(p=0.2),
    v2.ColorJitter(brightness=0.2, contrast=0.2, saturation=0.2, hue=0.1),
    v2.RandomRotation(degrees=15),
    v2.RandomAffine(degrees=0, translate=(0.1, 0.1)),
    v2.RandomGrayscale(p=0.1),

    normalize_transforms
])

val_transforms = normalize_transforms

train_dataset = MapDataset(train_dataset, transform=train_transforms)
val_dataset = MapDataset(val_dataset, transform=val_transforms)

train_dataloader = DataLoader(
    train_dataset, batch_size=s.dataset["batch_size"], shuffle=True, num_workers=cpu_count)
val_dataloader = DataLoader(
    val_dataset, batch_size=s.dataset["batch_size"],  num_workers=cpu_count)

Files already downloaded and verified


In [5]:
callbacks = [
    EarlyStoppingCallback(min_val_accuracy=90.0, accuracy_diff=5.0, wait_epochs=5),
    # OverfitCallback(limit_batches=1, batch_size=10, max_epochs=500, augument_data=False)
]

In [6]:
model = ResNet18(num_classes=100)
summary(model, input_size=(train_dataset[0][0].shape), batch_size=s.dataset["batch_size"], device="cpu")

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1          [512, 64, 32, 32]           1,728
       BatchNorm2d-2          [512, 64, 32, 32]             128
              ReLU-3          [512, 64, 32, 32]               0
         ConvBlock-4          [512, 64, 32, 32]               0
            Conv2d-5         [512, 128, 32, 32]          73,728
       BatchNorm2d-6         [512, 128, 32, 32]             256
              ReLU-7         [512, 128, 32, 32]               0
         ConvBlock-8         [512, 128, 32, 32]               0
            Conv2d-9         [512, 128, 32, 32]         147,456
      BatchNorm2d-10         [512, 128, 32, 32]             256
             ReLU-11         [512, 128, 32, 32]               0
        ConvBlock-12         [512, 128, 32, 32]               0
           Conv2d-13         [512, 128, 32, 32]         147,456
      BatchNorm2d-14         [512, 128,

In [7]:
module = ResNetModule(model)

optimizer = optim.AdamW(
    params=module.model.parameters(),
    weight_decay=s.optimizer["weight_decay"] if s.optimizer["weight_decay"] else 0.01
)

try:
    if s.lr_scheduler["name"] == "OneCycleLR":
        lr_scheduler = torch.optim.lr_scheduler.OneCycleLR(
            optimizer=optimizer,
            max_lr=s.lr_scheduler["max_lr"],
            epochs=s.max_epochs,
            steps_per_epoch=len(train_dataloader),
        )

        print(s.lr_scheduler["name"])
except TypeError:
    lr_scheduler = None
    print("lr_scheduler is None!")

OneCycleLR


In [8]:
# module.model, optimizer = load_from_checkpoint(
#     path="../logs/wandb/offline-run-20241215_132918-77n093vj/checkpoints/best.pt",
#     model=module.model,
#     optimizer=optimizer
# )

# summary(module.model, input_size=(train_dataset[0][0].shape), batch_size=s.dataset["batch_size"], device="cpu")

In [9]:
trainer = Trainer(
    module=module,
    logger=logger,
    optimizer=optimizer,
    callbacks=callbacks,
    logs_path=logs_path,
    fast_dev_run=s.fast_dev_run,
    limit_batches=s.limit_batches,
    lr_scheduler=lr_scheduler,
    save_checkpoint_type="best_val",
    num_workers=cpu_count
)

Using device: cuda!


In [10]:
try:
    trainer.fit(train_dataloader, val_dataloader)
except KeyboardInterrupt as e:
    print("Run stopped!")
finally:
    wandb.finish()

wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.
wandb: Currently logged in as: sampath017. Use `wandb login --relogin` to force relogin


Time per epoch: 64.01 seconds
Epoch: 0, train_accuracy: 4.36, val_accuracy: 5.49, lr: 0.0004
Epoch: 1, train_accuracy: 7.46, val_accuracy: 7.91, lr: 0.0005
Epoch: 2, train_accuracy: 10.11, val_accuracy: 11.88, lr: 0.0008
Epoch: 3, train_accuracy: 12.70, val_accuracy: 16.38, lr: 0.0013
Epoch: 4, train_accuracy: 15.73, val_accuracy: 16.92, lr: 0.0020
Epoch: 5, train_accuracy: 18.85, val_accuracy: 18.90, lr: 0.0028
Epoch: 6, train_accuracy: 21.57, val_accuracy: 19.63, lr: 0.0037
Epoch: 7, train_accuracy: 24.92, val_accuracy: 23.49, lr: 0.0047
Epoch: 8, train_accuracy: 28.20, val_accuracy: 23.76, lr: 0.0057
Epoch: 9, train_accuracy: 31.06, val_accuracy: 22.17, lr: 0.0067
Epoch: 10, train_accuracy: 34.13, val_accuracy: 31.74, lr: 0.0076
Epoch: 11, train_accuracy: 36.36, val_accuracy: 33.38, lr: 0.0084
Epoch: 12, train_accuracy: 39.17, val_accuracy: 38.61, lr: 0.0091
Epoch: 13, train_accuracy: 41.53, val_accuracy: 38.89, lr: 0.0096
Epoch: 14, train_accuracy: 44.00, val_accuracy: 45.08, lr: 0

epoch,▁▁▁▁▁▂▂▂▂▂▃▃▃▃▃▄▄▄▄▄▄▄▅▅▅▆▆▆▆▆▇▇▇▇▇█████
epoch_train_accuracy,▁▁▁▂▂▂▃▃▃▃▄▄▄▅▅▅▅▅▅▆▆▆▆▆▆▇▇▇▇▇▇▇████████
epoch_train_loss,██▇▇▇▆▆▅▅▅▄▄▄▄▄▃▃▃▃▃▃▃▂▂▂▂▂▂▂▂▁▁▁▁▁▁▁▁▁▁
epoch_val_accuracy,▁▁▂▂▂▃▃▃▄▄▅▅▅▅▆▆▆▆▇▇▇▇▇▇▇▇▇█████████████
epoch_val_loss,██▇▆▆▆▆▆▄▄▄▃▃▃▂▂▂▂▂▂▂▂▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
lr,▁▁▁▁▂▃▄▄▄▄▆▇█████████▇▇▇▇▆▆▅▅▄▄▃▃▃▃▂▂▁▁▁
step_train_accuracy,▁▂▂▂▂▃▃▄▄▄▅▅▅▅▅▅▅▅▅▆▅▆▆▆▆▆▆▆▇▇▇█▇███████
step_train_loss,██▇▇▇▆▆▆▅▅▅▅▅▄▄▄▃▃▃▃▃▃▃▂▃▂▃▃▂▂▂▂▁▁▁▁▁▁▁▁
step_val_accuracy,▁▂▂▂▃▄▅▅▅▅▅▆▅▆▆▆▆▆▆▆▆▆▇▇▇▇▇▇▇▇▇▇▇▇█▇▇▇▇█
step_val_loss,█▇▇▆▆▇▇▇▆▆▅▄▃▃▃▂▂▂▂▂▂▂▂▂▂▂▁▂▂▂▂▁▂▂▂▂▂▂▁▁
training_step,▁▁▁▂▂▂▂▂▂▃▃▃▄▄▄▄▄▄▅▅▅▅▅▅▅▅▅▆▆▆▇▇▇▇▇▇▇███


[Metrics](https://api.wandb.ai/links/sampath017/iwrrziwg)